In [1]:
#define initial variable
biting_rate= 0.3 #per day
prob_of_spread_to_human= 0.088 #probility of infecting malaria from mosquitoes to human
prob_of_infect_to_mos = 0.087 #probility of infecting malaria from human to mosquitoes
mos_birth_rate = 4 #mosquitoes birth rate
mos_mortal_rate = 0.1 #mosquitoes mortality rate
mos_carry_capacity = 500000 #mosquitoes capacity of the area
extentic_incubation_period = 7
seasonal_prefix = 7/100
seasonal_suffix = 5.3/100
frequent_seasonality = 2.7/1000
num_cluster = 8 #number of cluster
sim_date = 365 #simulation date

In [2]:
# define class
class Agent:
    #init constructor
    class_counter= 0
    def __init__(self,health_stage,cluster,occupation,timer=0):
        self.id= Agent.class_counter
        self.health_stage = health_stage
        self.cluster = cluster
        self.occupation = occupation
        self.risk = 0.0
        self.timer = timer
        self.destination = None
        Agent.class_counter+=1

class Cluster:
    def __init__(self,cluster_number,mos_s,mos_l,mos_i):
        self.cluster_number = cluster_number
        self.area_risk = 0.0
        self.mos_s = mos_s
        self.mos_l = mos_l
        self.plus_latent = 0
        self.mos_i = mos_i
        self.population= 0
        self.patient=0

In [3]:
# create array in each cluster to keep each agent status
cluster_field = [[] for i in range(0,num_cluster)]
cluster_field
# create array to keep all agent
all_agent = []
#define function for adding agent to each cluster
def agentCreator(cluster,size,health_stage,occupation,groupList,timer=0):
    for j in range(0,size):
        all_agent.append(Agent(health_stage=health_stage,occupation=occupation,cluster=cluster,timer=timer))

        cluster_field[cluster].append(all_agent[-1])

        cluster_detail[cluster].population += 1
        if health_stage == "Infected":
            cluster_detail[cluster].patient += 1
            groupList.append(cluster_field[cluster][-1])

In [4]:
#initial state value
#initial each cluster state (define the cluste detail ex. cluster number & number of mosquitoes in each state)
cluster_detail = []
cluster_detail.append(Cluster(cluster_number=0,mos_s=1000,mos_l=0,mos_i=0))
cluster_detail.append(Cluster(cluster_number=1,mos_s=500,mos_l=0,mos_i=0))
cluster_detail.append(Cluster(cluster_number=2,mos_s=100,mos_l=0,mos_i=0))
cluster_detail.append(Cluster(cluster_number=3,mos_s=100,mos_l=0,mos_i=0))
cluster_detail.append(Cluster(cluster_number=4,mos_s=100,mos_l=0,mos_i=0))
cluster_detail.append(Cluster(cluster_number=5,mos_s=100,mos_l=0,mos_i=0))
cluster_detail.append(Cluster(cluster_number=6,mos_s=100,mos_l=0,mos_i=0))
cluster_detail.append(Cluster(cluster_number=7,mos_s=100,mos_l=0,mos_i=0))
# define agent group
agent_officer=[]
agent_farmer=[]
agent_unemployed=[]
# create agent and apply to each cluster
for i in range(0,num_cluster):
    agentCreator(i,size= 100,health_stage="Healthy", occupation="Officer", groupList=agent_officer)
    agentCreator(i,size= 100,health_stage="Healthy", occupation="Farmer", groupList=agent_farmer)
    agentCreator(i,size= 100,health_stage="Healthy", occupation="Unemployed", groupList=agent_unemployed)
# add infected agent to cluster 4
agentCreator(4,size= 100,health_stage="Infected", occupation="Unemployed", groupList=agent_unemployed,timer=20)

In [5]:
#define probility function and travel rule
from numpy.random import choice
import random
#route_choice = [[0 for x in range(num_cluster)] for y in range(num_cluster)]
from numpy.random import choice
def takeodds(probability): #funciton for taking the probility ex. roll for getting malaria
    return random.random() < probability
def route(agent): #function for define the traveling route of the agent
    draw = [None]
    if agent.cluster== 0: #origin cluster where agent start
        draw = choice([1,2,3,4,5,7],1,True,[0.1,0.4,0.25,0.1,0.1,0.05]) # choice of destination & probility
    elif agent.cluster== 1:
        draw = choice([0,7],1,True,[0.4,0.6])
    elif agent.cluster== 2:
        draw = choice([0,1,3,7],1,True,[0.5,0.3,0.1,0.1])
    elif agent.cluster== 3:
        draw = choice([2,7],1,True,[0.4,0.6])
    elif agent.cluster== 4:
        draw = choice([0,7],1,True,[0.5,0.5])
    elif agent.cluster== 5:
        draw = choice([0,7],1,True,[0.5,0.5])
    elif agent.cluster== 6:
        draw = [7]
    elif agent.cluster== 7:
        draw = choice([2,4,6],1,True,[0.5,0.3,0.2,])
    # define new route for new cluster
    # elif agent.cluster== 8:
    #     draw = choice([2,4,6],1,True,[0.5,0.3,0.2,])

    else: draw=[None] #
    return draw[0] #return the choosen destination of the agent

def travelOdds(agent): # function for roll the odds of travel in each occupation
    destination= None
    if agent.occupation == "Officer":
        if takeodds(0.3): # probility of traveling of Officer is 0.3
            return route(agent)
    elif agent.occupation == "Farmer":
        if takeodds(0.1): # probility of traveling of Farmer is 0.1
            return route(agent)
    elif agent.occupation == "Unemployed":
        if takeodds(0.1): # probility of traveling of Unemployed is 0.1
            return route(agent)
# create infecting and incubating function
def icubatingOdds():  #create function for roll probility turning the agent from healthy into incubating stage
    for i in range(0,len(all_agent)): #interate all agent
            if all_agent[i].health_stage == "Healthy": # look for agent with susceptible stage
                if takeodds(all_agent[i].risk): # roll the odds with the risk of each agent, If true agent got infected
                    all_agent[i].health_stage = "Incubating"
                    all_agent[i].timer = random.randrange(7,31) # random the period of intection

def infectingOdds(): #create function for tracking timer( countdown incubating & infecting period) and changing agent state
    for i in range(0,len(all_agent)):
        if all_agent[i].health_stage != "Healthy":
            all_agent[i].timer -=1
            if all_agent[i].health_stage == "Incubating" and all_agent[i].timer == 0 :
                if takeodds(1):
                    all_agent[i].health_stage = "Infected" #changing agent stage to infected
                    cluster_detail[all_agent[i].cluster].patient +=1 #tracking the nunber of patient in the cluster
                    if takeodds(0.5): #odd for each maralia species recovery date
                        all_agent[i].timer = 14
                    else: all_agent[i].timer = 20
                else: all_agent[i].health_stage = "Healthy"

            if all_agent[i].health_stage == "Infected" and all_agent[i].timer == 0 :
                all_agent[i].health_stage = "Healthy"
                cluster_detail[all_agent[i].cluster].patient -= 1 #tracking the nunber of patient in the cluster
#create mobility funciton for agent
def allocateAgent(agent,newcluster): # function for agent (remove agent from old cluster and append to new cluster)
    cluster_detail[agent.cluster].population -=1
    cluster_detail[newcluster].population +=1
    if agent.health_stage == "Infected": #check for infected agent for updating in cluster detail
        cluster_detail[agent.cluster].patient -= 1
        cluster_detail[newcluster].patient +=1
    cluster_field[newcluster].append(agent)
    index = 999 #placeholder for finding the agnet id
    for i in range(0,len(cluster_field[agent.cluster])): #loop for find agent id in old cluster
        if(agent.id == cluster_field[agent.cluster][i].id):
            index = i
    cluster_field[agent.cluster].pop(index) #remove agent from old cluster
    cluster_field[newcluster][-1].cluster = newcluster #update the cluster number of traveling agent

In [6]:
#create function for counting each healthstage of inputted group
count_prop = []
def count_health_stage(agent_group):
    count_healthy = 0
    count_incubate = 0
    count_infect = 0
    for i in range(0,len(agent_group)):
        if agent_group[i].health_stage == "Healthy":
            count_healthy += 1
        elif agent_group[i].health_stage == "Incubating":
            count_incubate += 1
        elif agent_group[i].health_stage == "Infected":
            count_infect += 1
    count_prop.append([count_healthy,count_incubate,count_infect])

    return [count_healthy,count_incubate,count_infect]
#create the function for updating the proportion of agent state in each cluster in dataframe
def update_pop_track_df(df):
    for cluster in range(0,len(cluster_field)):
        cluster_prop = count_health_stage(cluster_field[cluster])
        pop_track_df = pd.concat([pop_track_df,pd.DataFrame.from_records([{"Date":date,"Cluster": cluster,"Healthy":cluster_prop[0],"Incubating":cluster_prop[1],"Infected":cluster_prop[2],"M_Suscept":cluster_detail[i].mos_s,"M_Latent":cluster_detail[i].mos_l,"M_Infected":cluster_detail[i].mos_i,"Risk":cluster_detail[i].area_risk}])],ignore_index=True)
# run the simulation
import copy
import math
import pandas as pd

# create array to keep tracking each stage of simulation
pop_track_df = pd.DataFrame(columns=["Date","Cluster","Healthy","Incubating","Infected","M_Suscept","M_Latent","M_Infected","Risk"])
stage=[]
cluster_stage=[]
stage=[copy.deepcopy(cluster_field)]
cluster_stage=[copy.deepcopy(cluster_detail)]
import pandas as pd
df = pd.DataFrame(columns=["Date","Healthy","Incubating","Infected"])

travel_history = pd.DataFrame(columns=["agentID","occupation","date","start","destination","health_stage"])



#calculate risk of all agent for first time
for i in range(0,len(all_agent)):
    health_stage_temp = count_health_stage(cluster_field[all_agent[i].cluster])
    #risk = 1-(pow((1-prob_of_spread_to_human),biting_rate))*infected_mos/population
    all_agent[i].risk = biting_rate*prob_of_spread_to_human*(cluster_detail[all_agent[i].cluster].mos_i/cluster_detail[all_agent[i].cluster].population)*(health_stage_temp[0]/cluster_stage[0][all_agent[i].cluster].population)
    cluster_detail[all_agent[i].cluster].area_risk = all_agent[i].risk
#update detail of each cluster on first date
for cluster in range(0,len(cluster_field)):
    cluster_prop = count_health_stage(cluster_field[cluster])
    pop_track_df = pd.concat([pop_track_df,pd.DataFrame.from_records([{"Date":0,"Cluster": cluster,"Healthy":cluster_prop[0],"Incubating":cluster_prop[1],"Infected":cluster_prop[2],"M_Suscept":copy.copy(cluster_detail[cluster].mos_s),"M_Latent":cluster_detail[cluster].mos_l,"M_Infected":cluster_detail[cluster].mos_i,"Risk":cluster_detail[cluster].area_risk}])],ignore_index=True)
for date in range(0,sim_date): # start simulation
    # for agent in agent_infect_farmer:
    #     print(agent.timer)
    #define array for traveller
    traveller = []
    #count prop from all of Cluster
    count_health = count_health_stage(all_agent)

    df = pd.concat([df, pd.DataFrame.from_records([{"Date":date,"Healthy":count_health[0],"Incubating":count_health[1],"Infected":count_health[2]}])], ignore_index=True)
    infectingOdds()
    for i in range(0,len(all_agent)): #loop all agent to make them decide whether to travel to somewhere
        destination=travelOdds(all_agent[i])
        if destination != None:
            travel_history = pd.concat([travel_history, pd.DataFrame.from_records([{"agentID":all_agent[i].id,"occupation":all_agent[i].occupation,"date":date+1,"start":all_agent[i].cluster,"destination":destination,"health_stage": all_agent[i].health_stage}])], ignore_index=True)
            allocateAgent(all_agent[i],destination)

    for i in range(0,len(all_agent)): #loop all agent to calculate the odds of infection
        health_stage_temp = count_health_stage(cluster_field[all_agent[i].cluster])
        #risk = 1-(pow((1-prob_of_spread_to_human),biting_rate))*infected_mos/population
        all_agent[i].risk = biting_rate*prob_of_spread_to_human*(cluster_detail[all_agent[i].cluster].mos_i/cluster_detail[all_agent[i].cluster].population) *(health_stage_temp[0]/cluster_stage[0][all_agent[i].cluster].population)
        cluster_detail[all_agent[i].cluster].area_risk = all_agent[i].risk

    icubatingOdds() #taking odds of every agent for infecting malaria
    #-------update mosquetoes stage--------------------------------------
    print("date",date)
    for i in range(0,len(cluster_detail)):
        temp_ms = copy.copy(cluster_detail[i].mos_s)
        temp_ml = copy.copy(cluster_detail[i].mos_l)
        temp_mi = copy.copy(cluster_detail[i].mos_i)
        n_m = temp_mi+temp_ms+temp_ml
        cluster_detail[i].plus_latent = biting_rate*prob_of_infect_to_mos*temp_ms*(cluster_detail[i].patient/cluster_detail[i].population)
        #------------------------------------mos susceptible stage----------------------------------------
        # debugger for latent stage
        # if(temp_ml < 0 and date >=7): #condition for first 7 day
        #     print("s=",temp_ms,"l=",temp_ml,"i=",temp_mi)
        #     print("ml",temp_ml,"+=",biting_rate*prob_of_infect_to_mos*temp_ms*(cluster_detail[i].patient/cluster_detail[i].population),"-=",cluster_stage[date-extentic_incubation_period][i].plus_latent)
        # # print("exp",math.exp(-mos_mortal_rate*extentic_incubation_period))
        #     print("infected_human",cluster_stage[date-extentic_incubation_period if date >= 7 else 0][i].patient,"Pop",cluster_stage[date-extentic_incubation_period if date >= 7 else 0][i].population)

        #------------------- Mosquitoes susceptible population increment
        cluster_detail[i].mos_s += mos_birth_rate*(n_m)*(1-(n_m/mos_carry_capacity))*(seasonal_prefix-seasonal_suffix*math.sin(2*math.pi*frequent_seasonality*date))
        #------------------- Mosquitoes susceptible population decrement
        cluster_detail[i].mos_s -= mos_mortal_rate*temp_ms
        cluster_detail[i].mos_s -= biting_rate*prob_of_infect_to_mos*temp_ms*(cluster_detail[i].patient/cluster_detail[i].population)
        #mos latent stage
        # print("bitr",biting_rate,"mosinfect",prob_of_infect_to_mos,"mos_sus",temp_ms,"patient",cluster_detail[i].patient,"pop",cluster_detail[i].population,"cal",biting_rate*prob_of_infect_to_mos*temp_ms*(cluster_detail[i].patient/cluster_detail[i].population))
        #------------------- Mosquitoes Latent increment
        cluster_detail[i].mos_l += biting_rate*prob_of_infect_to_mos*temp_ms*(cluster_detail[i].patient/cluster_detail[i].population)
        #------------------- Mosquitoes susceptible decrement
        cluster_detail[i].mos_l -= mos_mortal_rate*temp_ml
        if(date >= extentic_incubation_period):
            cluster_detail[i].mos_l -= math.exp(-mos_mortal_rate*extentic_incubation_period)*biting_rate*prob_of_infect_to_mos*cluster_stage[date-extentic_incubation_period][i].plus_latent
        #mos infected stage

        #------------------- Mosquitoes susceptible increment
        if date >= extentic_incubation_period:
            cluster_detail[i].mos_i += math.exp(-mos_mortal_rate*extentic_incubation_period)*biting_rate*prob_of_infect_to_mos*cluster_stage[date-extentic_incubation_period][i].plus_latent
        #------------------- Mosquitoes susceptible decrement
        cluster_detail[i].mos_i -= mos_mortal_rate*temp_mi


    for cluster in range(0,len(cluster_field)): #loop for tracking the cluster status on each date of simulation
        cluster_prop = count_health_stage(cluster_field[cluster])
        pop_track_df = pd.concat([pop_track_df,pd.DataFrame.from_records([{"Date":date+1,"Cluster": cluster,"Healthy":cluster_prop[0],"Incubating":cluster_prop[1],"Infected":cluster_prop[2],"M_Suscept":cluster_detail[cluster].mos_s,"M_Latent":cluster_detail[cluster].mos_l,"M_Infected":cluster_detail[cluster].mos_i,"Risk":cluster_detail[cluster].area_risk}])],ignore_index=True)
    cluster_stage.append(copy.deepcopy(cluster_detail))
    stage.append(copy.deepcopy(cluster_field))

C:\Users\HP\AppData\Local\Temp\ipykernel_16764\4059101780.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pop_track_df = pd.concat([pop_track_df,pd.DataFrame.from_records([{"Date":0,"Cluster": cluster,"Healthy":cluster_prop[0],"Incubating":cluster_prop[1],"Infected":cluster_prop[2],"M_Suscept":copy.copy(cluster_detail[cluster].mos_s),"M_Latent":cluster_detail[cluster].mos_l,"M_Infected":cluster_detail[cluster].mos_i,"Risk":cluster_detail[cluster].area_risk}])],ignore_index=True)


date 0
date 1
date 2
date 3
date 4
date 5
date 6
date 7
date 8
date 9
date 10
date 11
date 12
date 13
date 14
date 15
date 16
date 17
date 18
date 19
date 20
date 21
date 22
date 23
date 24
date 25
date 26
date 27
date 28
date 29
date 30
date 31
date 32
date 33
date 34
date 35
date 36
date 37
date 38
date 39
date 40
date 41
date 42
date 43
date 44
date 45
date 46
date 47
date 48
date 49
date 50
date 51
date 52
date 53
date 54
date 55
date 56
date 57
date 58
date 59
date 60
date 61
date 62
date 63
date 64
date 65
date 66
date 67
date 68
date 69
date 70
date 71
date 72
date 73
date 74
date 75
date 76
date 77
date 78
date 79
date 80
date 81
date 82
date 83
date 84
date 85
date 86
date 87
date 88
date 89
date 90
date 91
date 92
date 93
date 94
date 95
date 96
date 97
date 98
date 99
date 100
date 101
date 102
date 103
date 104
date 105
date 106
date 107
date 108
date 109
date 110
date 111
date 112
date 113
date 114
date 115
date 116
date 117
date 118
date 119
date 120
date 121
date 122
dat

In [7]:
#convert the dataframe of travel record into csv
travel_history.to_csv('travel_history.csv', encoding='utf-8', index=False)

In [8]:
# read the cluster location file and merge with the travel record to call the latitude and longtitude of each cluster
cluster_loc = pd.read_csv('cluster_location.csv')
pop_track_df=pop_track_df.merge(cluster_loc, left_on='Cluster',right_on='id',  how='left')
pop_track_df=pop_track_df.drop(['id', 'name'], axis=1)
pop_track_df.to_csv('cluster_detail.csv', encoding='utf-8', index=False)
pop_track_df

,Date,Cluster,Healthy,Incubating,Infected,M_Suscept,M_Latent,M_Infected,Risk,lat,lon
0,0,0,300,0,0,1000,0,0,0.000000e+00,17.338760,98.106891
1,0,1,300,0,0,500,0,0,0.000000e+00,17.113713,98.400515
2,0,2,300,0,0,100,0,0,0.000000e+00,17.472902,98.026301
3,0,3,300,0,0,100,0,0,0.000000e+00,17.227205,98.225675
4,0,4,300,0,100,100,0,0,0.000000e+00,17.212100,97.768400
...,...,...,...,...,...,...,...,...,...,...,...
2923,365,3,178,0,0,341239.647766,0.0,0.0,2.055534e-21,17.227205,98.225675
2924,365,4,246,0,0,341239.647766,0.0,0.0,2.006570e-19,17.212100,97.768400
2925,365,5,50,0,0,341239.647766,0.0,0.0,3.394329e-22,16.711983,98.566046
2926,365,6,125,0,0,341239.647766,0.0,0.0,2.143129e-21,17.338300,97.667000
